In [93]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.stats import hypergeom
from scipy.stats import binned_statistic as binsta
from scipy.special import logsumexp
from util import *
import palettable as pal
clrx = pal.cartocolors.qualitative.Prism_10.mpl_colors
clr = tuple(x for n,x in enumerate(clrx) if n in [1,2,4,5,6])
clr2 = pal.cartocolors.sequential.agSunset_7.mpl_colors
import matplotlib.gridspec as gridspec
import matplotlib.patches as patches

import plotly.graph_objects as go
import plotly.tools as tls
from plotly.offline import plot, iplot, init_notebook_mode
from IPython.core.display import display, HTML
init_notebook_mode(connected = True)
config={'showLink': False, 'displayModeBar': False}

# CCP, the Coupon Collector's Problem
def ccp_sample(c,pool=60):
    return len(set(np.random.choice(pool,c)))

# Draw overlap
def nab_sample(s,na,nb,pool=60):
    sa = np.random.hypergeometric(s,pool-s,na)
    nab = np.random.hypergeometric(sa,pool-sa,nb)
    return nab

# Overlap between two PCRs of depth c and overlap s
def pcr_sample(c,s):
    na = ccp_sample(c)
    nb = ccp_sample(c)
    return nab_sample(s,na,nb),na,nb

# Draw na and nb samples from two populations of size pool_a and pool_b, with true overlap s
# and return empirical overlap between na and nb
# note that this is basically the same as nab_sample, but with two different size pools!
def nab_sample_unequal(s,na,nb,pool_a,pool_b):
    sa = np.random.hypergeometric(s,pool_a-s,na)
    nab = np.random.hypergeometric(sa,pool_b-sa,nb)
    return nab


def p_ccp(c, pool=60):
    p = np.zeros([c+1,pool+1])
    p[0,0] = 1;
    for row in range(1,c+1):
        for k in range(1,np.min([row+2,pool+1])):
            p[row,k] = p[row-1,k]*k/pool + p[row-1,k-1]*(1-(k-1)/pool)
    return p[-1,:]

def p_overlap(na,nb,nab,pool=60):
    p_s = np.zeros(pool+1)
    # reference: hypergeom.pmf(outcome, Total, hits, Draws, loc=0)
    for s in np.arange(pool+1):
        # p_sa is the probability that we'd get sa from the overlap (s), just in na draws of a
        p_sa = hypergeom.pmf(np.arange(pool+1),pool,s,na)
        # p_nab_given_sa is the probability of getting that nab, given sa
        p_nab_given_sa = hypergeom.pmf(nab,pool,np.arange(pool+1),nb)
        p_s[s] = np.dot(p_sa,p_nab_given_sa)
    return p_s/np.sum(p_s)

def e_overlap(na,nb,nab,pool=60):
    p_s = p_overlap(na,nb,nab,pool=pool)
    return np.dot(np.arange(pool+1),p_s)


def credible_interval(na,nb,nab,pct=90,pool=60):
    p_s = p_overlap(na,nb,nab,pool=pool)
    cdf = np.cumsum(p_s)
    ccdf = np.flipud(np.cumsum(np.flipud(p_s)))
    # adjust for fractions vs percents; put everything as a fraction
    if pct > 1:
        pct = pct/100
    cutoff = (1-pct)/2
    # get the lower bound. 
    # it's the first index at which cdf ≥ cutoff
    try:
        lower = np.where(cdf >= cutoff)[0][0]
    except IndexError:
        lower = 0
    # get the upper bound
    # it's the first index at which ccdf ≥ 0.05
    try:
        upper = np.where(ccdf >= cutoff)[0][-1]
    except IndexError:
        upper=pool
    expectation = np.dot(np.arange(pool+1),p_s)
    # Sanity and indexing check: uncomment this line to see true tail probability ≤ 0.05
    # print([cdf[lower-1],(1-ccdf[upper+1])])
    return lower,expectation,upper


def p_nab_given_c(s,c,pool=60):
    pna = p_ccp(c)
    pnb = p_ccp(c)
    nas = np.arange(1,len(pna))
    nbs = np.arange(1,len(pnb))
    p_gen = np.zeros([pool+1,pool+1,pool+1])
    for na in nas:
        p_sa = hypergeom.pmf(np.arange(pool+1),pool,s,na)
        for nb in nbs:
            pna_pnb = pna[na] * pnb[nb]
            for nab in range(0,np.minimum(na,nb)):
                p_nab_given_sa = hypergeom.pmf(nab,pool,np.arange(pool+1),nb)
                p_nab_given_s = np.dot(p_sa,p_nab_given_sa)
                p_gen[na,nb,nab] = p_nab_given_s * pna_pnb
    return p_gen

def p_shat_given_sc(s,c,shat,pool=60):
    masses = p_nab_given_c(s,c,pool=pool)
    if np.sum(masses)<0.99:
        print('Swapping to Monte Carlo')
        return p_shat_given_sc_montecarlo(s,c,shat,pool=pool)
    hist = binsta(np.ravel(shat),np.ravel(masses),statistic='sum',bins=(np.arange(pool+2)-0.5))
    return hist

def p_shat_given_sc_montecarlo(s,c,shat,pool=60,n_mc=int(1e5)):
    masses = np.zeros([pool+1,pool+1,pool+1])
    for ii in range(n_mc):
        nab,na,nb = pcr_sample(c,s)
        masses[na,nb,nab] += 1
    hist = binsta(np.ravel(shat),np.ravel(masses/n_mc),statistic='sum',bins=(np.arange(pool+2)-0.5))
    return hist

def compute_all_estimates(pool=60):
    shat = np.zeros([pool+1,pool+1,pool+1])
    for na in range(1,pool+1):
        for nb in range(1,pool+1):
            for nab in range(0,np.minimum(na+1,nb+1)):
                shat[na,nb,nab] = e_overlap(na,nb,nab,pool=pool)
    return shat

def p_overlap_unequal(na,nb,nab,pool_a,pool_b):
    # all loops are in terms of pool_a, which is assumed to be ≤ pool_b. 
    p_s = np.zeros(pool_a+1)
    # reference: hypergeom.pmf(outcome, Total, hits, Draws, loc=0)
    for s in np.arange(pool_a+1):
        # p_sa is the probability that we'd get sa from the overlap (s), just in na draws of a
        p_sa = hypergeom.pmf(np.arange(pool_a+1),pool_a,s,na)
        # p_nab_given_sa is the probability of getting that nab, given sa
        p_nab_given_sa = hypergeom.pmf(nab,pool_b,np.arange(pool_a+1),nb)
        p_s[s] = np.dot(p_sa,p_nab_given_sa)
    return p_s/np.sum(p_s)

def e_overlap_unequal(na,nb,nab,pool_a,pool_b):
    # TODO. Code expects that pool_b > pool_a...
    p_s = p_overlap_unequal(na,nb,nab,pool_a,pool_b)
    return np.dot(np.arange(pool_a+1),p_s)


In [99]:
# shat = compute_all_estimates(pool=60)
# np.save('shat_60.npy',shat)
shat = np.load('shat_60.npy')
planted_new = []
recovered_new = []
recovered_pts_new = []
pool=60
reps=4

# Temps
x1 = 30
# Calculate list for 30-50 steps 
for i in range(0, 7, 1):
    recovered = np.zeros([6,reps,pool+1])
    planted = np.zeros([6,reps,pool+1])
    recovered_pts = np.zeros([6,reps,pool+1])
    
    # Calculate for row 30-32, 33-35, 36-38, 39-41, 42-44, 45-47, 48-50
    nas = [x1, x1+1, x1+2]
    for idxa,na in enumerate(nas):
        nb = na
        for s in np.arange(0,pool+1,1):
            for rep in range(3):
                nab = nab_sample(s,na,nb)
                planted[idxa,rep,s] = s
                recovered[idxa,rep,s] = e_overlap(na,nb,nab)
                recovered_pts[idxa,rep,s] = 2*nab/(na+nb)
                
    # Flatten arrays 
    for index in range(3):
        tmp1 = planted[index,:,:]
        tmp2 = recovered[index,:,:]
        tmp3 = recovered_pts[index,:,:]

        planted_new.append( tmp1.flatten() )
        recovered_new.append( tmp2.flatten() )
        recovered_pts_new.append( tmp3.flatten() )
    x1 = x1 + 3
    
#     # Append to main 
#     for index in range(3):
#         planted_new.append(planted1[index])
#         recovered_new.append(recovered1[index])
#         recovered_pts_new.append(recovered_pts1[index])

In [86]:
# fig3 = go.Figure()
# z = np.zeros(60)
# d = np.arange(1,60)

# fig3.add_trace(go.Scatter(x = d , y = d, line = dict(width=1, dash='dash', color = 'rgb(0,0,0)'), name = 'Reference', showlegend = False))

# fig3.add_trace(go.Scatter(x=planted_new[12], y=recovered_new[12],
#                     mode='markers',
#                     name = 'BRO',
#                   marker=dict(color='rgba(129,105,177,0.85)')))

# fig3.add_trace(go.Scatter(x=planted_new[12], y=pool*recovered_pts_new[12],
#                     mode='markers',
#                     name = 'S',
#                     marker_symbol = "x",
#                     marker=dict(color='gray')))

# fig3.update_layout(plot_bgcolor='rgb(255,255,255)',xaxis_title="True overlap s",
#     yaxis_title='Estimated overlap \u015D', legend=dict(x=.85, y=.1))
# fig3.update_xaxes(ticks = 'outside', showline=True, linecolor='black')
# fig3.update_yaxes(ticks = 'outside', showline=True, linecolor='black')

# plot(fig3, filename = 'fig3.html', config = config)
# iplot(fig3)

In [101]:
len(planted_new)

21

In [102]:
recovered_new[0] == recovered_new[2]

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,

In [103]:
data = []

trace0 = go.Scatter(x = d , 
                    y = d, 
                    line = dict(width=1, dash='dash', color = 'rgb(0,0,0)'), 
                    name = 'Reference', 
                    showlegend = False)

colors = ['rgba(129,105,177,0.85)', 'rgba(172,110,191,0.85)','rgba(215,128,193,0.85)'] 
current_color = 0

for index in range(len(planted_new)):
    
    if index > 5:
        current_color = 1
        
    elif index > 15:
        current_color = 2
        
    
    trace1 = go.Scatter(x=planted_new[index], 
                        y=recovered_new[index],
                        mode='markers',
                        name = 'BRO',
                        visible = False,
                        marker=dict(color=colors[current_color]))
    
    trace2 = go.Scatter(x=planted_new[index], 
                        y=pool*recovered_pts_new[index],
                        mode='markers',
                        name = 'S',
                        visible = False,
                        marker_symbol = "x",
                        marker=dict(color='gray'))
    
    data.append(trace1)
    data.append(trace2)

data.append(trace0)

In [104]:
len(data)

43

In [106]:
# Toggle first trace to be visible
data[0]['visible'] = True
data[1]['visible'] = True


# Create steps and slider
steps = []
for i in range(0, len(planted_new)):
    step = dict(
        method = 'restyle',  
        args = ['visible', [False]*43],
        label = str(i+30)) 
    
    step['args'][1][i*2] = True 
    step['args'][1][i*2+1] = True 
    step['args'][1][-1] = True
    steps.append(step)

sliders = [dict(
    active = 0,
    currentvalue = {'prefix':
                    "<i>Sample size</i>: <b>"},
    pad = {"t": 80, "b": 10},
    steps = steps
)]

# Setup the layout of the figure
layout = go.Layout(title = "Figure 3: Bayesian overlap",
                   title_x = 0.5, 
                   plot_bgcolor='rgb(255,255,255)',
                   xaxis_title="True overlap s",
                   yaxis_title='Estimated overlap \u015D', 
                   legend=dict(x=.85, y=.1),
                   width = 800, 
                   height = 450,
                   sliders = sliders,
                   font = dict(size = 10),
                   margin=go.layout.Margin(l=50,
                                         r=50,
                                         b=60,
                                         t=35))

# Plot function saves as html or with ipplot
fig_enhanced = dict(data=data, layout=layout)
plot(fig_enhanced, filename = 'fig3_enhanced.html', config = config)
# display(HTML('fig3_enhanced.html'))

'fig3_enhanced.html'